<a href="https://colab.research.google.com/github/PortugalJip2022/Portugal2022/blob/main/20230529_%E6%B0%B4%E8%B0%B7%E7%B7%A8%E9%9B%86_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Flask
project_name = "sum_app" #@param {type:"string"}


In [ ]:
#@title サーバー
port = "8000" #@param {type:"string"}
host = True #@param {type:"boolean"}
from google.colab.output import eval_js
host = eval_js(f"google.colab.kernel.proxyPort({str(port)})")


In [ ]:
#フォルダ作成
!rm -rf $project_name
!mkdir $project_name && mkdir $project_name/templates && mkdir $project_name/static

In [ ]:
#@title sum.py(json)形式で入力データ保存)

%%writefile sum.py

from flask import Flask, redirect ,request,render_template,jsonify

import pandas as pd
import numpy as np
import json
import requests

import pickle
import traceback

app = Flask(__name__)

#モデル読み込み
with open('model.pkl','rb') as f:
  model = pickle.load(f)

@app.route("/")
def check():
    return render_template("sum.html")

@app.route("/output", methods=["POST"])
def output():
  try:
    _A = int(request.json["_A"])  #年齢
    _B = int(request.json["_B"])  #残高

    # データ保存
    w_data = {'age_60':_A, 'balance_5000':_B}
    with open('w_data.json','w') as f:
      json.dump(w_data,f)

    # jsonファイル参照
    with open('w_data.json','r') as f:
      data = json.load(f)

    #dataframeに変換
    df = pd.DataFrame([data])

    #あてはめ
    p = model.predict(df)[0]
    p_n=round(p,4)*100

    #結果データ保存
    data['p']=p
    with open('w_data_result.json','w') as f:
      json.dump(data,f)

    data = {"result":p,"result":p_n, "_A":_A, "_B":_B, "msg":"%s％の確率で契約しそう"%(p_n)}
    return jsonify(ResultSet=json.dumps(data)), 200
    data = {"result":p}
    return jsonify(ResultSet=json.dumps(data)), 200

  except Exception as err:
    print(err)
    trace = traceback.format_exc()
    return {"error":trace},500

    #data = {"error":"err"}
    #return jsonify(ResultSet=json.dumps(data)), 500

if __name__ == "__main__":
    import sys
    args = sys.argv
    app.run(debug=True, port=args[1])

Overwriting sum.py


In [ ]:
#@title sum.html

%%writefile $project_name/templates/sum.html


<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title>sum app</title>
</head>
<body>
    年齢<input type="text" value="0" name="_A" /> 残高 <input type="text" value="0" name="_B" /> <button id="run">計算</button><br/>
    <h2>結果</h2>
    <script src="static/post.js"></script>
</body>
</html>



Writing sum_app/templates/sum.html


In [ ]:
#@title post.js

%%writefile $project_name/static/post.js


fetch("https://code.jquery.com/jquery-3.4.1.min.js").then(r=>{return r.text()}).then(t=>eval(t)).then(()=>{
	$=jQuery;
    document.getElementById("run").addEventListener("click",
        function(){
            values=[...document.querySelectorAll("input")].map(_=>_.value);
            $.ajax({
                url:"/output",
                type:"POST",
                data:JSON.stringify({"_A":values[0],"_B":values[1]}),
                contentType:"application/json"
            }).done(data=>{
                $(document.body).append(JSON.parse(data.ResultSet).msg,"<br/>");
            }).fail(data=>{
                $(document.body).append("error","<br/>")
            });
        },false);
});


Overwriting sum_app/static/post.js


In [ ]:
print('①が起動してから、↓のurlにアクセスする')
print(host)

①が起動してから、↓のurlにアクセスする
https://1nprj39t077-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [ ]:
#①
!python $project_name/sum.py $port

 * Serving Flask app 'sum'
 * Debug mode: on
 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 381-105-533
127.0.0.1 - - [02/Aug/2023 03:42:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2023 03:42:20] "GET /static/post.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2023 03:42:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Aug/2023 03:42:21] "POST /output HTTP/1.1" 200 -


↓ここから

In [ ]:
# pip install -U pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
# pip install pyinstaller

  Obtaining dependency information for pyinstaller from https://files.pythonhosted.org/packages/54/8b/eecf92dbbd8846ce8382506b39af182e92dc470de2cff2b54dfcd4e5def7/pyinstaller-5.13.0-py3-none-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyinstaller-hooks-contrib>=2021.4 from https://files.pythonhosted.org/packages/d9/2f/675f01ccfccabf18cfb211feb7280cc13cfc1253eda4a6ffee32b56e1bdb/pyinstaller_hooks_contrib-2023.6-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 655.0/655.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.3/274.3 kB 11.8 MB/s eta 0:00:00


In [ ]:
# %%writefile $project_name/sum.py
%%writefile sum.py

import subprocess
import sys
import os

from flask import Flask, redirect ,request,render_template,jsonify

import json
import requests

app = Flask(__name__)

@app.route("/")
def check():
    return render_template("sum.html")

@app.route("/output", methods=["POST"])
def output():
    try:
        _A = int(request.json["_A"])
        _B = int(request.json["_B"])
        res = _A + _B
        data = {"result":res, "_A":_A, "_B":_B, "msg":"%s + %s = %s"%(_A,_B,res)}
        return jsonify(ResultSet=json.dumps(data)), 200
    except Exception as err:
        print(err)
        data = {"error":"err"}
        return jsonify(ResultSet=json.dumps(data)), 500

if __name__ == "__main__":
    import sys
    args = sys.argv
    app.run(debug=True, port=args[1])

Overwriting sum_app/sum.py


In [ ]:
import sys
sys.setrecursionlimit(5000)

The 'pathlib' package is an obsolete backport of a standard library package and is incompatible with PyInstaller. Please remove this package  using conda remove then try again.

In [ ]:
# pip install pyinstaller         #バージョン設定しないとexe化コードが動かない
# pip install pyinstaller==5.1
# !pip install pyinstaller==4.0
!pip install pyinstaller==4.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pyinstaller
    Found existing installation: pyinstaller 4.0
    Uninstalling pyinstaller-4.0:
      Successfully uninstalled pyinstaller-4.0


In [ ]:
#!pyinstaller --onefile app.py
!pyinstaller --onefile app.py --exclude-module _bootlocale

182 INFO: PyInstaller: 4.0
182 INFO: Python: 3.10.12
184 INFO: Platform: Linux-5.15.109+-x86_64-with-glibc2.35
184 INFO: wrote /content/app.spec
185 INFO: UPX is not available.
186 INFO: Extending PYTHONPATH with paths
['/content', '/content']
pygame 2.5.0 (SDL 2.28.0, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
319 INFO: checking Analysis
319 INFO: Building Analysis because Analysis-00.toc is non existent
319 INFO: Initializing module dependency graph...
324 INFO: Caching module graph hooks...
333 WARNING: Several hooks defined for module 'pygame'. Please take care they do not conflict.
336 INFO: Analyzing base_library.zip ...
5539 INFO: Processing pre-find module path hook distutils from '/usr/local/lib/python3.10/dist-packages/PyInstaller/hooks/pre_find_module_path/hook-distutils.py'.
5591 INFO: distutils: retargeting to non-venv dir '/usr/lib/python3.10'
6949 INFO: Caching module dependency graph...
7051 INFO: running Analysis Analysis-00

In [ ]:
/dist/sum

NameError: ignored

In [ ]:
# !pyinstaller --onefile --exclude-module pathlib sum.py

The 'pathlib' package is an obsolete backport of a standard library package and is incompatible with PyInstaller. Please remove this package (located in /usr/local/lib/python3.10/dist-packages) using
    "/usr/bin/python3" -m pip uninstall pathlib
then try again.


In [ ]:
#@title app.py(お試し版）

%%writefile app.py

def add_numbers(a, b):
  return a + b

if __name__ == '__main__':
  import sys
  a = 20  #int(sys.argv[1])
  b = 5 #int(sys.argv[2])
  result = add_numbers(a, b)
  # sys.stdout.write(str(result))


Overwriting app.py


In [ ]:
#@title app.py

%%writefile app.py

from flask import Flask, redirect ,request,render_template,jsonify

import pandas as pd
import numpy as np
import json
import requests

import pickle
import traceback


app = Flask(__name__)

#モデル読み込み
with open('model.pkl','rb') as f:
  model = pickle.load(f)

# @app.route("/")
# def check():
#    return render_template("sum.html")

# @app.route("/output", methods=["POST"])

def output(a,b):

  # _A = int(request.json["_A"])  #年齢
  # _B = int(request.json["_B"])  #残高

  _A = int(a)  #年齢
  _B = int(b)  #残高

  # データ保存
  w_data = {'age_60':_A, 'balance_5000':_B}
  # with open('w_data.json','w') as f:
  #   json.dump(w_data,f)

  # jsonファイル参照
  # with open('w_data.json','r') as f:
  #   data = json.load(f)

#dataframeに変換
  # df = pd.DataFrame([data])
  df = pd.DataFrame([w_data])

  #あてはめ
  p = model.predict(df)[0]
  p_n=round(p,4)*100

  return  p_n

  #結果データ保存
  # data['p']=p
  # with open('w_data_result.json','w') as f:
  #   json.dump(data,f)

  # data = {"result":p,"result":p_n, "_A":_A, "_B":_B, "msg":"%s％の確率で契約しそう"%(p_n)}
  # return jsonify(ResultSet=json.dumps(data)), 200
  # data = {"result":p}
  # return jsonify(ResultSet=json.dumps(data)), 200

  # except Exception as err:
  #   print(err)
  #   trace = traceback.format_exc()
  #   return {"error":trace},500

    #data = {"error":"err"}
    #return jsonify(ResultSet=json.dumps(data)), 500

if __name__ == "__main__":
    import sys

    a=int(sys.argv[1])
    b=int(sys.argv[2])
    result =output(a,b)
    # sys.stdout.write(str(result))

    # args = sys.argv
    # app.run(debug=True, port=args[1])

Overwriting app.py


In [ ]:
#test
_A = 20 #年齢
_B = 300  #残高

# データ保存
w_data = {'age_60':_A, 'balance_5000':_B}

In [ ]:
import pandas as pd
df = pd.DataFrame([w_data])
df


,age_60,balance_5000
0,20,300


In [ ]:
%%writefile app_1.py

print("hello world!")

Writing app_1.py


In [ ]:
# !pyinstaller --onefile app_1.py
!pyinstaller --onefile app_1.py --exclude-module _bootlocale

225 INFO: PyInstaller: 4.8
225 INFO: Python: 3.10.12
226 INFO: Platform: Linux-5.15.109+-x86_64-with-glibc2.35
227 INFO: wrote /content/app_1.spec
228 INFO: UPX is not available.
229 INFO: Extending PYTHONPATH with paths
['/content']
Traceback (most recent call last):
  File "<string>", line 3, in <module>
  File "/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py", line 29, in <module>
    import zipfile
  File "/usr/lib/python3.10/zipfile.py", line 19, in <module>
    import pathlib
  File "/usr/local/lib/python3.10/dist-packages/pathlib.py", line 10, in <module>
    from collections import Sequence
ImportError: cannot import name 'Sequence' from 'collections' (/usr/lib/python3.10/collections/__init__.py)
283 INFO: checking Analysis
283 INFO: Building Analysis because Analysis-00.toc is non existent
283 INFO: Initializing module dependency graph...
286 INFO: Caching module graph hooks...
289 INFO: Analyzing base_library.zip ...
6845 INFO: Processing pre-find module pat

In [ ]:
%%writefile app_t.py

from flask import Flask
import sys
port = 6000 #int(sys.argv[1]) #コマンド引数でPORT番号を取得
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'hello_world'

def main():
    app.debug = True
    app.run(port=port)

if __name__ == '__main__':
    main()

Overwriting app_t.py


In [ ]:
from google.colab import output
PORT = 6000
PATH = ''
# output.serve_kernel_port_as_window(6000, path=PATH)

# サーバー起動
# !python3 /content/app_t.py $PORT
!python3 /content/app_t.py

 * Serving Flask app 'app_t'
 * Debug mode: on
 * Running on http://127.0.0.1:6000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 381-105-533
